## Import the necessary modules

In [1]:
import mysql.connector as connector
import os
import datetime as dt
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import logging

## Create a logger

In [2]:
logger = logging.getLogger("[Database Structures and Management with MySQL]")
logging.basicConfig(filename='management-with-mysql.log', encoding='utf-8', level=logging.DEBUG, format='%(asctime)s ==> %(message)s', datefmt='%m/%d/%Y %I:%M:%S')

## Establish a connection

In [3]:
dbconfig={"user":"root", "password":os.environ["MYSQL_PASSWORD"]}
connection=connector.connect(port=33061, **dbconfig)
print("Connection between MySQL and Python is established.")
logger.info("Connection between MySQL and Python is established.")

Connection between MySQL and Python is established.


## Create a cursor object

In [4]:
cursor = connection.cursor()
print("Cursor created to communicate with the MySQL using Python.")
logger.info("Cursor created to communicate with the MySQL using Python.")

Cursor created to communicate with the MySQL using Python.


## Create Database 

In [5]:
drop_database_query = """DROP DATABASE IF EXISTS db_meta_dsm_mysql"""
cursor.execute(create_database_query)
logger.info("Dropping Database if it already exists.")

create_database_query: str = """CREATE DATABASE IF NOT EXISTS db_meta_dsm_mysql"""
print("Creating Database.")
logger.info("Creating Database.")
cursor.execute(create_database_query)
logger.info("Database created.")

Creating Database.


In [6]:
# Check to see that the database was created
list_of_databases: list = []
cursor.execute("SHOW DATABASES;")
databases = cursor.fetchall()
for database in databases:
    name: str = database[0]
    list_of_databases.append(name)
    if name == "db_meta_dsm_mysql":
        print("Database 'db_meta_dsm_mysql' was successfully created")
        logger.info("Database 'db_meta_dsm_mysql' was successfully created.")
print(list_of_databases)
logger.info(list_of_databases)

Database 'db_meta_dsm_mysql' was successfully created
['CVD', 'LittleLemonDB', 'PC', 'STAFF_LOCATIONS', 'db_Exercise', 'db_hr', 'db_learner', 'db_little_lemon', 'db_meta', 'db_meta_dsm_mysql', 'db_mysqladmin', 'db_world', 'global_super_store', 'information_schema', 'little_lemon', 'little_lemon_db', 'lucky_Shrub', 'mangata_gallo', 'mangata_jw_db', 'meta_db', 'mg_schema', 'mysql', 'performance_schema', 'sakila', 'sys']


In [7]:
# Set the new created database as the database to use
cursor.execute("USE db_meta_dsm_mysql")
print("Database 'db_meta_dsm_mysql' is set for use.")
logger.info("Database 'db_meta_dsm_mysql' is set for use.")

Database 'db_meta_dsm_mysql' is set for use.


## Show Tables

In [9]:
## Exepct an empty list to be returned 

cursor.execute("SHOW TABLES;")
results = cursor.fetchall()
for table in results:
    print(table[0]) 

## Create Table and populate with data

In [10]:
create_table_query = """CREATE TABLE IF NOT EXISTS tbl_orders (OrderID INT NOT NULL PRIMARY KEY, ClientID VARCHAR(10),  ProductID VARCHAR(10),  Quantity INT, Cost DECIMAL(6,2));"""
cursor.execute(create_table_query)
print("Table tbl_orders successfully created")
logger.info("Table tbl_orders successfully created")

# Check to that the table is indeed created
cursor.execute("SHOW TABLES;")
results = cursor.fetchall()
for table in results:
    print(table[0]) 

Table tbl_orders successfully created
tbl_orders


In [12]:
insert_into_tbl_order_query = """INSERT INTO tbl_orders (OrderID, ClientID, ProductID , Quantity, Cost) VALUES (1, "Cl1", "P1", 10, 500), (2, "Cl2", "P2", 5, 100), (3, "Cl3", "P3", 20, 800), (4, "Cl4", "P4", 15, 150),
(5, "Cl3", "P3", 10, 450), (6, "Cl2", "P2", 5, 800), (7, "Cl1", "P4", 22, 1200), (8, "Cl3", "P1", 15, 150), (9, "Cl1", "P1", 10, 500), (10, "Cl2", "P2", 5, 100);"""

# Populate Menu table
print("Inserting records into the tbl_orders table")
logger.info("Inserting records into the tbl_orders table")
cursor.execute(insert_into_tbl_order_query)
connection.commit()

Inserting records into the tbl_orders table


In [19]:
## Show all the records in the table
select_query = "SELECT * FROM tbl_orders;"
cursor.execute(select_query)
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names)
for result in results:
    print(result)

('OrderID', 'ClientID', 'ProductID', 'Quantity', 'Cost')
(1, 'Cl1', 'P1', 10, Decimal('500.00'))
(2, 'Cl2', 'P2', 5, Decimal('100.00'))
(3, 'Cl3', 'P3', 20, Decimal('800.00'))
(4, 'Cl4', 'P4', 15, Decimal('150.00'))
(5, 'Cl3', 'P3', 10, Decimal('450.00'))
(6, 'Cl2', 'P2', 5, Decimal('800.00'))
(7, 'Cl1', 'P4', 22, Decimal('1200.00'))
(8, 'Cl3', 'P1', 15, Decimal('150.00'))
(9, 'Cl1', 'P1', 10, Decimal('500.00'))
(10, 'Cl2', 'P2', 5, Decimal('100.00'))


# Filtering Task

## Task 1
Write a SQL statement to print all records of orders where the cost is $250 or less.

In [17]:
select_query = "SELECT * FROM tbl_orders WHERE Cost <= 250;"
cursor.execute(select_query)
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names)
for result in results:
    print(result)

('OrderID', 'ClientID', 'ProductID', 'Quantity', 'Cost')
(2, 'Cl2', 'P2', 5, Decimal('100.00'))
(4, 'Cl4', 'P4', 15, Decimal('150.00'))
(8, 'Cl3', 'P1', 15, Decimal('150.00'))
(10, 'Cl2', 'P2', 5, Decimal('100.00'))


## Task 2
Write a SQL statement to print all records of orders where the cost is between \\$50 and \\$750.

In [21]:
select_query = "SELECT * FROM tbl_orders WHERE Cost BETWEEN 50 and 750;"
cursor.execute(select_query)
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names)
for result in results:
    print(result)

('OrderID', 'ClientID', 'ProductID', 'Quantity', 'Cost')
(1, 'Cl1', 'P1', 10, Decimal('500.00'))
(2, 'Cl2', 'P2', 5, Decimal('100.00'))
(4, 'Cl4', 'P4', 15, Decimal('150.00'))
(5, 'Cl3', 'P3', 10, Decimal('450.00'))
(8, 'Cl3', 'P1', 15, Decimal('150.00'))
(9, 'Cl1', 'P1', 10, Decimal('500.00'))
(10, 'Cl2', 'P2', 5, Decimal('100.00'))


## Task 3
Write a SQL statement to print all records of orders that have been placed by the client with the id of Cl3 and where the cost of the order is more than $100.

In [22]:
select_query = "SELECT * FROM tbl_orders WHERE ClientID = 'Cl3' AND Cost > 100;"
cursor.execute(select_query)
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names)
for result in results:
    print(result)

('OrderID', 'ClientID', 'ProductID', 'Quantity', 'Cost')
(3, 'Cl3', 'P3', 20, Decimal('800.00'))
(5, 'Cl3', 'P3', 10, Decimal('450.00'))
(8, 'Cl3', 'P1', 15, Decimal('150.00'))


## Task 4
Write a SQL statement to print all records of orders that have a product id of p1 or p2 and the order quantity is more than 2. 

In [23]:
select_query = "SELECT * FROM tbl_orders WHERE ProductID IN ('P1', 'P2') AND Quantity > 2;"
cursor.execute(select_query)
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names)
for result in results:
    print(result)

('OrderID', 'ClientID', 'ProductID', 'Quantity', 'Cost')
(1, 'Cl1', 'P1', 10, Decimal('500.00'))
(2, 'Cl2', 'P2', 5, Decimal('100.00'))
(6, 'Cl2', 'P2', 5, Decimal('800.00'))
(8, 'Cl3', 'P1', 15, Decimal('150.00'))
(9, 'Cl1', 'P1', 10, Decimal('500.00'))
(10, 'Cl2', 'P2', 5, Decimal('100.00'))


# Joins

In [25]:
## Create additional tables
create_table_customers_query = """CREATE TABLE IF NOT EXISTS tbl_customers(CustomerID INT NOT NULL PRIMARY KEY, FullName VARCHAR(100) NOT NULL, PhoneNumber INT NOT NULL UNIQUE);"""
create_table_bookings_query = """CREATE TABLE IF NOT EXISTS tbl_bookings (BookingID INT NOT NULL PRIMARY KEY,  BookingDate DATE NOT NULL, TableNumber INT NOT NULL, NumberOfGuests INT NOT NULL CHECK (NumberOfGuests <= 8), 
CustomerID INT NOT NULL, FOREIGN KEY (CustomerID) REFERENCES tbl_customers (CustomerID) ON DELETE CASCADE ON UPDATE CASCADE); """

cursor.execute(create_table_customers_query)
print("Table tbl_customers successfully created")
logger.info("Table tbl_customers successfully created")
cursor.execute(create_table_bookings_query)
print("Table tbl_bookings successfully created")
logger.info("Table tbl_bookings successfully created")

Table tbl_customers successfully created
Table tbl_bookings successfully created


In [27]:
# Check to that the two additional tables were created in the database
cursor.execute("SHOW TABLES;")
results = cursor.fetchall()
assert len(results) == 3
for table in results:
    print(table[0]) 

tbl_bookings
tbl_customers
tbl_orders


In [30]:
## Populate the tables with Data
insert_into_tbl_customers = """INSERT INTO tbl_customers(CustomerID, FullName, PhoneNumber) VALUES (1, "Vanessa McCarthy", 0757536378), (2, "Marcos Romero", 0757536379), (3, "Hiroki Yamane", 0757536376), (4, "Anna Iversen", 0757536375), 
(5, "Diana Pinto", 0757536374);"""
logger.info("Inserting records into the tbl_customers table")
cursor.execute(insert_into_tbl_customers)
connection.commit()

insert_into_tbl_bookings = """INSERT INTO tbl_bookings (BookingID, BookingDate, TableNumber, NumberOfGuests, CustomerID) VALUES (10, '2021-11-11', 7, 5, 1), (11, '2021-11-10', 5, 2, 2), (12, '2021-11-10', 3, 2, 4);"""
logger.info("Inserting records into the tbl_bookings table")
cursor.execute(insert_into_tbl_bookings)
connection.commit()

## Task 1
Write an INNER JOIN SQL statement to combine the full name and the phone number of each customer from the Customers table with the related booking date and 'number of guests' from the Bookings table. 

In [99]:
inner_join_query = """SELECT FullName, PhoneNumber, BookingDate, NumberOfGuests FROM tbl_customers
INNER JOIN tbl_bookings
ON tbl_customers.CustomerID = tbl_bookings.CustomerID;"""
cursor.execute(inner_join_query)
results = cursor.fetchall()
table_column_names = cursor.column_names
# print(table_column_names)
print("+-------------------+-------------+--------------+----------------+")
print(f"| {table_column_names[0]:18}| {table_column_names[1]} | {table_column_names[2]}  | {table_column_names[3]} |")
print("+-------------------+-------------+--------------+----------------+")
for result in results:
    print(f"| {result[0]:18}|  {result[1]:8}  | {result[2]}   | {result[3]: 6}         |")
print("+-------------------+-------------+--------------+----------------+")

+-------------------+-------------+--------------+----------------+
| FullName          | PhoneNumber | BookingDate  | NumberOfGuests |
+-------------------+-------------+--------------+----------------+
| Vanessa McCarthy  |  757536378  | 2021-11-11   |      5         |
| Marcos Romero     |  757536379  | 2021-11-10   |      2         |
| Anna Iversen      |  757536375  | 2021-11-10   |      2         |
+-------------------+-------------+--------------+----------------+


## Task 2
Write a LEFT JOIN SQL statement to view the customer id from Customers table and the related booking id from the Bookings table.

In [101]:
left_join_query = """SELECT tbl_customers.CustomerID, BookingID FROM tbl_customers
LEFT JOIN tbl_bookings
ON tbl_customers.CustomerID = tbl_bookings.CustomerID;"""
cursor.execute(left_join_query)
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names)
for result in results:
    print(result)

('CustomerID', 'BookingID')
(5, None)
(4, 12)
(3, None)
(1, 10)
(2, 11)


# Group By

In [103]:
create_new_order_table = """CREATE TABLE orders_tbl(OrderID INT, Department VARCHAR(100), OrderDate DATE, OrderQty INT, OrderTotal INT, PRIMARY KEY(OrderID));"""
cursor.execute(create_new_order_table)
print("Table orders_tbl successfully created")
logger.info("Table orders_tbl successfully created")

Table orders_tbl successfully created


In [104]:
insert_into_new_table = """INSERT INTO orders_tbl VALUES(1,'Lawn Care','2022-05-05',12,500),(2,'Decking','2022-05-22',150,1450),(3,'Compost and Stones','2022-05-27',20,780),(4,'Trees and Shrubs','2022-06-01',15,400),(5,'Garden Decor','2022-06-10',2,1250),
(6,'Lawn Care','2022-06-10',12,500),(7,'Decking','2022-06-25',150,1450),(8,'Compost and Stones','2022-05-29',20,780),(9,'Trees and Shrubs','2022-06-10',15,400),(10,'Garden Decor','2022-06-10',2,1250),(11,'Lawn Care','2022-06-25',10,400), 
(12,'Decking','2022-06-25',100,1400),(13,'Compost and Stones','2022-05-30',15,700),(14,'Trees and Shrubs','2022-06-15',10,300),(15,'Garden Decor','2022-06-11',2,1250),(16,'Lawn Care','2022-06-10',12,500),
(17,'Decking','2022-06-25',150,1450),(18,'Trees and Shrubs','2022-06-10',15,400),(19,'Lawn Care','2022-06-10',12,500),(20,'Decking','2022-06-25',150,1450),(21,'Decking','2022-06-25',150,1450);"""


logger.info("Inserting records into the orders_tbl table")
cursor.execute(insert_into_new_table)
connection.commit()

In [105]:
cursor.execute("SELECT * FROM orders_tbl")
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names)
for result in results:
    print(result)

('OrderID', 'Department', 'OrderDate', 'OrderQty', 'OrderTotal')
(1, 'Lawn Care', datetime.date(2022, 5, 5), 12, 500)
(2, 'Decking', datetime.date(2022, 5, 22), 150, 1450)
(3, 'Compost and Stones', datetime.date(2022, 5, 27), 20, 780)
(4, 'Trees and Shrubs', datetime.date(2022, 6, 1), 15, 400)
(5, 'Garden Decor', datetime.date(2022, 6, 10), 2, 1250)
(6, 'Lawn Care', datetime.date(2022, 6, 10), 12, 500)
(7, 'Decking', datetime.date(2022, 6, 25), 150, 1450)
(8, 'Compost and Stones', datetime.date(2022, 5, 29), 20, 780)
(9, 'Trees and Shrubs', datetime.date(2022, 6, 10), 15, 400)
(10, 'Garden Decor', datetime.date(2022, 6, 10), 2, 1250)
(11, 'Lawn Care', datetime.date(2022, 6, 25), 10, 400)
(12, 'Decking', datetime.date(2022, 6, 25), 100, 1400)
(13, 'Compost and Stones', datetime.date(2022, 5, 30), 15, 700)
(14, 'Trees and Shrubs', datetime.date(2022, 6, 15), 10, 300)
(15, 'Garden Decor', datetime.date(2022, 6, 11), 2, 1250)
(16, 'Lawn Care', datetime.date(2022, 6, 10), 12, 500)
(17, 'Dec

## Task 1
Write a SQL SELECT statement to group all records that have the same order date.

In [109]:
cursor.execute("SELECT OrderDate FROM orders_tbl GROUP BY OrderDate")
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names[0])
for result in results:
    print(result[0])

OrderDate
2022-05-05
2022-05-22
2022-05-27
2022-06-01
2022-06-10
2022-06-25
2022-05-29
2022-05-30
2022-06-15
2022-06-11


## Task 2
Write a SQL SELECT statement to retrieve the number of orders placed on the same day.

In [114]:
cursor.execute("SELECT OrderDate, Count(OrderQty) AS OrderQuantity FROM orders_tbl GROUP BY OrderDate")
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names[0],    table_column_names[1])
for result in results:
    print(result[0], result[1])

OrderDate OrderQuantity
2022-05-05 1
2022-05-22 1
2022-05-27 1
2022-06-01 1
2022-06-10 7
2022-06-25 6
2022-05-29 1
2022-05-30 1
2022-06-15 1
2022-06-11 1


## Task 3
Write a SQL SELECT statement to retrieve the total order quantities placed by each department.

In [130]:
cursor.execute("SELECT Department, Sum(OrderQty) AS OrderQuantity FROM orders_tbl GROUP BY Department")
results = cursor.fetchall()
table_column_names = cursor.column_names
print(f"{table_column_names[0]:20} {table_column_names[1]}")
for result in results:
    print(f"{result[0]:20} {result[1]:6}")

Department           OrderQuantity
Lawn Care                58
Decking                 850
Compost and Stones       55
Trees and Shrubs         55
Garden Decor              6


## Task 4
Write a SQL SELECT statement to retrieve the number of orders placed on the same day between the following dates: 1st June 2022 and 30th June 2022.

In [140]:
cursor.execute("SELECT OrderDate, Count(OrderQty) AS OrderQuantity FROM orders_tbl WHERE OrderDate BETWEEN '2022-06-01' AND '2022-06-30' GROUP BY OrderDate")
results = cursor.fetchall()
table_column_names = cursor.column_names
print(f"{table_column_names[0]:15} {table_column_names[1]}")
for result in results:
    print(f"{result[0]} {result[1]:10}")

OrderDate       OrderQuantity
2022-06-01          1
2022-06-10          7
2022-06-25          6
2022-06-15          1
2022-06-11          1


# Replace Into
How the REPLACE INTO statement works
The REPLACE statement checks whether the intended data record's unique key value already exists in the table before inserting it as a new record or updating it. 

The REPLACE INTO statement attempts to insert a new record or modify an existing record. In both cases, it checks whether the unique key of the proposed record already exists in the table. Suppose a value of NO or FALSE is returned. In that case, the REPLACE statement inserts the record similar to the INSERT INTO statement.

Suppose the key value already exists in the table (in other words, a duplicate key). In that case, the REPLACE statement deletes the existing record of data and replaces it with a new record of data. This happens regardless of whether you use the first or the second REPLACE statement syntax.

```sql
REPLACE INTO table_name (column1name, column2name, ...) 
VALUES (value1, value2, ...);
```


```sql
REPLACE INTO table_name SET column1name = value, column2name = value, ... ;
```

In [142]:
cursor.execute("SELECT * FROM tbl_orders")
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names)
for result in results:
    print(result)

('OrderID', 'ClientID', 'ProductID', 'Quantity', 'Cost')
(1, 'Cl1', 'P1', 10, Decimal('500.00'))
(2, 'Cl2', 'P2', 5, Decimal('100.00'))
(3, 'Cl3', 'P3', 20, Decimal('800.00'))
(4, 'Cl4', 'P4', 15, Decimal('150.00'))
(5, 'Cl3', 'P3', 10, Decimal('450.00'))
(6, 'Cl2', 'P2', 5, Decimal('800.00'))
(7, 'Cl1', 'P4', 22, Decimal('1200.00'))
(8, 'Cl3', 'P1', 15, Decimal('150.00'))
(9, 'Cl1', 'P1', 10, Decimal('500.00'))
(10, 'Cl2', 'P2', 5, Decimal('100.00'))


## Task 1
Write a SQL REPLACE statement that inserts two new orders with the following details:

Order 11 data:
- OrderID = 11, 
- ClientID = "Cl2", 
- ProductID = "P3", 
- Quantity = 50, 
- Cost = 5000

Order 12 data:
- OrderID = 12, 
- ClientID = "Cl2", 
- ProductID = "P2", 
- Quantity = 25, 
- Cost = 1000

In [144]:
replace_query = """REPLACE INTO tbl_orders (OrderID, ClientID, ProductID, Quantity, Cost) 
VALUES (11, 'Cl2', 'P3', 50, 5000), (12, 'Cl2', 'P2', 25, 1000);"""

cursor.execute(replace_query)
connection.commit()

In [145]:
cursor.execute("SELECT * FROM tbl_orders")
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names)
for result in results:
    print(result)

('OrderID', 'ClientID', 'ProductID', 'Quantity', 'Cost')
(1, 'Cl1', 'P1', 10, Decimal('500.00'))
(2, 'Cl2', 'P2', 5, Decimal('100.00'))
(3, 'Cl3', 'P3', 20, Decimal('800.00'))
(4, 'Cl4', 'P4', 15, Decimal('150.00'))
(5, 'Cl3', 'P3', 10, Decimal('450.00'))
(6, 'Cl2', 'P2', 5, Decimal('800.00'))
(7, 'Cl1', 'P4', 22, Decimal('1200.00'))
(8, 'Cl3', 'P1', 15, Decimal('150.00'))
(9, 'Cl1', 'P1', 10, Decimal('500.00'))
(10, 'Cl2', 'P2', 5, Decimal('100.00'))
(11, 'Cl2', 'P3', 50, Decimal('5000.00'))
(12, 'Cl2', 'P2', 25, Decimal('1000.00'))


## Task 2
It was noticed that the cost of order number 11 is \\\$5000. This is a mistake. The order must cost \\$500. You must help them to change it to \\$500 by writing a relevant REPLACE statement. 

In [146]:
replace_query_set = """REPLACE INTO tbl_orders SET OrderID = 11, ClientID = 'Cl2', ProductID = 'P3', Quantity = 50, Cost = 500;"""

cursor.execute(replace_query_set)
connection.commit()

In [147]:
cursor.execute("SELECT * FROM tbl_orders")
results = cursor.fetchall()
table_column_names = cursor.column_names
print(table_column_names)
for result in results:
    print(result)

('OrderID', 'ClientID', 'ProductID', 'Quantity', 'Cost')
(1, 'Cl1', 'P1', 10, Decimal('500.00'))
(2, 'Cl2', 'P2', 5, Decimal('100.00'))
(3, 'Cl3', 'P3', 20, Decimal('800.00'))
(4, 'Cl4', 'P4', 15, Decimal('150.00'))
(5, 'Cl3', 'P3', 10, Decimal('450.00'))
(6, 'Cl2', 'P2', 5, Decimal('800.00'))
(7, 'Cl1', 'P4', 22, Decimal('1200.00'))
(8, 'Cl3', 'P1', 15, Decimal('150.00'))
(9, 'Cl1', 'P1', 10, Decimal('500.00'))
(10, 'Cl2', 'P2', 5, Decimal('100.00'))
(11, 'Cl2', 'P3', 50, Decimal('500.00'))
(12, 'Cl2', 'P2', 25, Decimal('1000.00'))
